In [123]:
import numpy as np
import pandas as pd

naics sectors: https://www.census.gov/programs-surveys/economic-census/year/2022/guidance/understanding-naics.html
isic sectors: https://unstats.un.org/unsd/classifications/Econ/Download/In%20Text/ISIC_Rev_4_publication_English.pdf

In [124]:
naics_sectors = {
    'Agriculture, Forestry, Fishing and Hunting': ['11'],
    'Mining, Quarrying, and Oil and Gas Extraction': ['21'],
    'Utilities': ['22'],
    'Construction': ['23'],
    'Manufacturing': ['31', '32', '33'],
    'Wholesale Trade': ['42'],
    'Retail Trade': ['44','45'],
    'Transportation and Warehousing': ['48', '49'],
    'Information': ['51'],
    'Finance and Insurance': ['52'],
    'Real Estate and Rental and Leasing': ['53'],
    'Professional, Scientific, and Technical Services': ['54'],
    'Management of Companies and Enterprises': ['55'],
    'Administrative and Support and Waste Management and Remediation Services': ['56'],
    'Educational Services': ['61'],
    'Health Care and Social Assistance': ['62'], 
    'Arts, Entertainment, and Recreation': ['71'],
    'Accommodation and Food Services': ['72'],
    'Other Services': ['81'],
    'Public Administration': ['92']
}


In [125]:
isic_sectors_df = pd.read_csv('ISIC_SECTORS.csv')

isic_sectors = {}

for i, row in isic_sectors_df.iterrows():
    
    subs = [int(x) for x in row[1].split('–')]
    if len(subs) > 1:
        subs = np.arange(subs[0], subs[1] + 1)
    subs = ['0' + str(x) if len(str(x)) < 2 else str(x) for x in subs]

    for s in subs:
        isic_sectors[s] = row[0]

In [126]:
naics_isic_df = pd.read_csv('NAICS_ISIC.csv', dtype={'NAICS2012Code': str, 'ISIC4Code': str})

In [127]:
naics_isic_df['ISICSec'] = naics_isic_df['ISIC4Code'].map(lambda x: isic_sectors[x[0:2]] )
naics_isic_df['NAICS_3'] = naics_isic_df['NAICS2012Code'].map(lambda x: x[0:3] if type(x) is str else None)
pd.set_option('display.max_rows', 200)
inter_map = naics_isic_df[['NAICS_3', 'ISICSec', 'NAICS2012Code']].groupby(['NAICS_3', 'ISICSec']).agg({'count'})
inter_map['best_match'] = None

matches = []
for code in list(np.unique(naics_isic_df['NAICS_3'].dropna())):
    best_match = inter_map.loc[code][('NAICS2012Code','count')].map(
        lambda x: x == max(inter_map.loc[code][('NAICS2012Code','count')])
    )
    for m in best_match: matches.append(m)

In [128]:
inter_map['best_match'] = matches
inter_map

NAICS2012Code best_match
                        count           
NAICS_3 ISICSec                         
111     A                  48       True
        C                   1      False
112     A                  26       True
113     A                   5       True
        C                   1      False
114     A                   7       True
115     A                  10       True
        M                   1      False
211     B                   4       True
212     B                  29       True
213     B                   5       True
        F                   3      False
221     D                  12       True
        E                   2      False
        H                   1      False
236     F                   9       True
237     F                  11       True
238     C                   3      False
        F                  29       True
        N                   1      False
311     A                   1      False
        B                   1      False
        C                  71       True
312     C                   8       True
        D                   1      False
313     C                  12       True
314     C                  15       True
315     C                  13       True
316     C                   6       True
321     C                  24       True
322     C                  25       True
323     C                   7       True
        N                   1      False
324     C                   7       True
325     C                  48       True
        N                   1      False
326     C                  22       True
327     C                  25       True
331     C                  27       True
332     C                  72       True
333     C                  98       True
334     C                  31       True
335     C                  25       True
336     C                  54       True
337     C                  22       True
339     C                  58       True
423     G                  66       True
424     G                  46       True
425     G                   8       True
441     C                   1      False
        G                  12       True
442     G                   5       True
        S                   1      False
443     C                   1      False
        G                   5       True
        S                   2      False
444     G                   8       True
        S                   1      False
445     G                  11       True
446     G                   5       True
447     G                   3       True
448     G                  10       True
        S                   2      False
451     G                   6       True
        S                   2      False
452     G                   6       True
453     C                   1      False
        G                  12       True
454     G                   9       True
481     H                   5       True
482     H                   3       True
483     H                   6       True
484     H                   6       True
485     H                  11       True
486     H                   4       True
487     H                   6       True
488     B                   1      False
        C                   3      False
        E                   1      False
        H                  20       True
        N                   4      False
491     H                   1       True
492     H                   2       True
493     H                   4       True
511     J                   8       True
512     J                  12       True
515     J                   8       True
517     J                   7       True
518     J                   1       True
        M                   1       True
519     J                  11       True
        R                   2      False
521     K                   1       True
522     K                  17       True
523     K                  12       Tru

In [129]:
naics_3_isicsec = {}

for i in inter_map[matches].index:
    if i[0] in naics_3_isicsec.keys():
        naics_3_isicsec[i[0]].append(i[1])
    else: naics_3_isicsec[i[0]] = [i[1]]

In [130]:
# ties include 

for key, val in naics_3_isicsec.items():
    naics_3_isicsec[key] = val[0]

naics_3_isicsec

{'111': 'A',
 '112': 'A',
 '113': 'A',
 '114': 'A',
 '115': 'A',
 '211': 'B',
 '212': 'B',
 '213': 'B',
 '221': 'D',
 '236': 'F',
 '237': 'F',
 '238': 'F',
 '311': 'C',
 '312': 'C',
 '313': 'C',
 '314': 'C',
 '315': 'C',
 '316': 'C',
 '321': 'C',
 '322': 'C',
 '323': 'C',
 '324': 'C',
 '325': 'C',
 '326': 'C',
 '327': 'C',
 '331': 'C',
 '332': 'C',
 '333': 'C',
 '334': 'C',
 '335': 'C',
 '336': 'C',
 '337': 'C',
 '339': 'C',
 '423': 'G',
 '424': 'G',
 '425': 'G',
 '441': 'G',
 '442': 'G',
 '443': 'G',
 '444': 'G',
 '445': 'G',
 '446': 'G',
 '447': 'G',
 '448': 'G',
 '451': 'G',
 '452': 'G',
 '453': 'G',
 '454': 'G',
 '481': 'H',
 '482': 'H',
 '483': 'H',
 '484': 'H',
 '485': 'H',
 '486': 'H',
 '487': 'H',
 '488': 'H',
 '491': 'H',
 '492': 'H',
 '493': 'H',
 '511': 'J',
 '512': 'J',
 '515': 'J',
 '517': 'J',
 '518': 'J',
 '519': 'J',
 '521': 'K',
 '522': 'K',
 '523': 'K',
 '524': 'K',
 '525': 'K',
 '531': 'L',
 '532': 'N',
 '533': 'N',
 '541': 'M',
 '551': 'K',
 '561': 'N',
 '562': 'E',

In [131]:
# compares to "Replication Packet/Converting SIC to NAICS/SIC_NAICS_BEA_allsec.do"

VW_groups = {
    'Agriculture': [111,112,113,114,115], # excluded
    'Mining': [212],
    'Oil/Gas': [211], # excluded
    'Mining Support': ['212'], # excluded
    'Util': [22], # 221 doesn't exist in NAICS codes in asset expenditures but in VW mapping
    'Const': [23],
    'Wood': [321],
    'Minerals': [327],
    'Primary Metals': [331],
    'Fabricated Metals': [332],
    'Machinery': [333],
    'Computers': [334],
    'Electrical': [335],
    'Vehicles': [3361,3362,3363],
    'Transport': [3364,3365,3366,3367,3368,3369],
    'Furniture': [337],
    'Misc Mfg': [339],
    'Food Mfg': [311,312],
    'Textile': [313,314],
    'Apparel': [315,316],
    'Paper': [322],
    'Printing': [323],
    'Petroleum': [324],
    'Chemical': [325],
    'Plastics': [326],
    'Wholesale Trade': [42],
    'Retail Trade': [44,45],
    'Transit/Warehouse': [48,49],
    'Info': [51],
    'Finance/Insurance': [52],
    'Real Estate': [531],
    'Rental': [532,533], # excluded
    'Prof/Tech': [54],
    'Mgmt': [55],
    'Admin': [561,562], # just 56 in the VW mapping
    'Educ': [61],
    'Health': [62],
    'Arts': [71],
    'Accom': [721],
    'Food Services': [722],
    'Other Services': [81]
}

In [132]:
VW_isicsec = {}
for key, vals in VW_groups.items():
    VW_isicsec[key] = []
    for v in vals:
        code = str(v)
        if len(code) == 3:
            VW_isicsec[key].append(naics_3_isicsec[code])
        elif len(code) == 2:
            matches = pd.Series(naics_3_isicsec.keys())[pd.Series(naics_3_isicsec.keys()).map(lambda x: x[0:2]) == code]
            for match in matches:
                VW_isicsec[key].append(naics_3_isicsec[match])
        elif len(code) == 4:
            VW_isicsec[key].append(naics_3_isicsec[code[0:3]])
    VW_isicsec[key] = list(np.unique(VW_isicsec[key]))

In [133]:
isicsec_VW = {}
for vw, sec in VW_isicsec.items():
    for sec in sec:
        if not sec in isicsec_VW.keys():
            isicsec_VW[sec] = [vw]
        else: isicsec_VW[sec].append(vw)

In [134]:
isicsec_VW

{'A': ['Agriculture'],
 'B': ['Mining', 'Oil/Gas', 'Mining Support'],
 'D': ['Util'],
 'F': ['Const'],
 'C': ['Wood',
  'Minerals',
  'Primary Metals',
  'Fabricated Metals',
  'Machinery',
  'Computers',
  'Electrical',
  'Vehicles',
  'Transport',
  'Furniture',
  'Misc Mfg',
  'Food Mfg',
  'Textile',
  'Apparel',
  'Paper',
  'Printing',
  'Petroleum',
  'Chemical',
  'Plastics',
  'Other Services'],
 'G': ['Wholesale Trade', 'Retail Trade'],
 'H': ['Transit/Warehouse'],
 'J': ['Info'],
 'K': ['Finance/Insurance', 'Mgmt'],
 'L': ['Real Estate'],
 'N': ['Rental', 'Admin'],
 'M': ['Prof/Tech'],
 'E': ['Admin'],
 'P': ['Educ'],
 'Q': ['Health'],
 'R': ['Arts'],
 'I': ['Accom', 'Food Services'],
 'S': ['Other Services'],
 'T': ['Other Services']}